In [1]:
!which python

/Users/davidmillard/Documents/Classes/QML/CSCI739-Quantum_Machine_Learning/project/venv/bin/python


In [2]:
import argparse
import math
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from lattice import RotatedSurfaceCode
from models import HLD_NN2_LSTM, HLD_NN1_LSTM
from datasets import HLDDatasetDepol, HLDDatasetCircuit

In [3]:
d=3
p=0.10
n_train=200_000
n_val=20_000
batch=1000
epochs=5000
seed=0
hidden=(16,4)
lr=1e-2

device = "mps"
code = RotatedSurfaceCode(d=d)
ds_tr = HLDDatasetDepol(code, p, n_train, seed)
ds_va = HLDDatasetDepol(code, p, n_val, seed+1)

In [4]:
ds_tr.__getitem__(2)

(tensor([[1., 0., 0., 0.]]), tensor([1., 0., 0., 0.]))

In [8]:

F = code.n_x_checks() + code.n_z_checks()
model = HLD_NN2_LSTM(input_dim=F, hidden_dims=hidden).to(device)
opt = torch.optim.Adam(model.parameters(), lr=lr)
# Paper used MSE with sigmoid outputs to approximate probabilities. :contentReference[oaicite:6]{index=6}
loss_fn = nn.MSELoss()

tr_loader = DataLoader(ds_tr, batch_size=batch, shuffle=True, drop_last=True)
va_loader = DataLoader(ds_va, batch_size=batch, shuffle=False, drop_last=False)

best_va = math.inf
patience_ctr = 0
for ep in range(1, 2):
    model.train()
    tot = 0.0
    for X, Y in tr_loader:
        X = X.to(device)  # [B, 1, F]
        Y = Y.to(device)  # [B, 4]
        opt.zero_grad()
        P = model(X)
        print(P.detach().cpu().numpy(), Y.detach().cpu().numpy())
        loss = loss_fn(P, Y)
        loss.backward()
        opt.step()
        tot += loss.item() * X.size(0)
    tr_loss = tot / len(ds_tr)

[[0.44734052 0.44762334 0.4080513  0.46922192]
 [0.44682688 0.4470457  0.4086686  0.4694764 ]
 [0.44734052 0.44762334 0.4080513  0.46922192]
 ...
 [0.44734052 0.44762334 0.4080513  0.46922192]
 [0.44734052 0.44762334 0.4080513  0.46922192]
 [0.44612217 0.44765028 0.40833813 0.4701365 ]] [[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]
[[0.45263216 0.44349653 0.40417084 0.46597114]
 [0.45271248 0.4433906  0.40441224 0.46548286]
 [0.45202827 0.4434931  0.4043507  0.4663717 ]
 ...
 [0.45263216 0.44349653 0.40417084 0.46597114]
 [0.45357165 0.44370183 0.40416983 0.4642769 ]
 [0.45263216 0.44349653 0.40417084 0.46597114]] [[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]
[[0.45791134 0.4395233  0.4003489  0.46264556]
 [0.4577826  0.4395394  0.4002271  0.4631537 ]
 [0.4577826  0.4395394  0.4002271  0.4631537 ]
 ...
 [0.45791134 0.4395233  0.4003489  0.46264556]
 [0.45791134 0.4395233  0.4003489  0.46